In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('result.csv')
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [3]:
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
class WaveDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class WaveClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(WaveClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [4]:
def load_data(file_path):
    data = pd.read_csv(file_path)
    features = data.drop(['label'], axis=1).values
    labels = data['label'].values
    
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    
    return features, labels, le

In [28]:
def train_model(model, train_loader, criterion, optimizer, device, num_epochs):
    model.train()
    losses = []
    for epoch in range(num_epochs):
        epoch_loss = 0
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            
            outputs = model(features)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        losses.append(avg_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [6]:
# Function to evaluate the model
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 加载数据
features, labels, le = load_data('result.csv')  # 替换为您的数据文件路径
    
# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [31]:

# 创建数据加载器
train_dataset = WaveDataset(X_train, y_train)
test_dataset = WaveDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 初始化模型
input_size = features.shape[1]
hidden_size = 50
num_classes = len(le.classes_)
model = WaveClassifier(input_size, hidden_size, num_classes).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 200
train_model(model, train_loader, criterion, optimizer, device, num_epochs)



Epoch [1/200], Loss: 2.0938
Epoch [2/200], Loss: 1.7911
Epoch [3/200], Loss: 0.9120
Epoch [4/200], Loss: 1.4924


KeyboardInterrupt: 

In [24]:
le

LabelEncoder()

In [30]:
# 评估模型
evaluate_model(model, test_loader, device)

Test Accuracy: 77.98%


77.98387096774194